# Multiscale MILP with Materials and Emission

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Example 2

In [1]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q


In [2]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

2025-10-19 22:23:40,791 [INFO] General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
2025-10-19 22:23:40,795 [INFO] ✔ Completed in 0.0004451274871826172 seconds
2025-10-19 22:23:40,797 [INFO] Binding consume in domain (solar, l, q)
2025-10-19 22:23:40,799 [INFO] ✔ Completed in 0.000392913818359375 seconds
2025-10-19 22:23:40,802 [INFO] General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
2025-10-19 22:23:40,804 [INFO] ✔ Completed in 0.0002257823944091797 seconds
2025-10-19 22:23:40,805 [INFO] Binding consume in domain (wind, l, y)
2025-10-19 22:23:40,807 [INFO] ✔ Completed in 0.00021195411682128906 seconds
2025-10-19 22:23:40,809 [INFO] General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
2025-10-19 22:23:40,811 [INFO] ✔ Completed in 0.00038123130798339844 seconds
2025-10-19 22:23:40,813 [INFO] Binding release in domain (power, l, q)
2025-10-19 22:23:40

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [3]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [4]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [5]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

2025-10-19 22:23:41,101 [INFO] General Resource Balance for cement in (l, y): initializing constraint, adding consume(cement, l, y)
2025-10-19 22:23:41,104 [INFO] ✔ Completed in 0.0003921985626220703 seconds
2025-10-19 22:23:41,105 [INFO] Binding consume in domain (cement, l, y)
2025-10-19 22:23:41,107 [INFO] ✔ Completed in 0.00021195411682128906 seconds


Next, we provide details of the use of cement across all operations 

In [6]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

2025-10-19 22:23:41,128 [INFO] General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, wf)
2025-10-19 22:23:41,133 [INFO] ✔ Completed in 0.0004208087921142578 seconds
2025-10-19 22:23:41,137 [INFO] General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, pv)
2025-10-19 22:23:41,140 [INFO] ✔ Completed in 0.00023317337036132812 seconds
2025-10-19 22:23:41,144 [INFO] General Resource Balance for cement in (l, y): adding use(cement, l, y, invcapacity, power.lii)
2025-10-19 22:23:41,147 [INFO] ✔ Completed in 0.0004458427429199219 seconds


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [7]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [8]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [9]:
m.network.locate(m.wf, m.pv, m.lii)

2025-10-19 22:23:41,291 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-19 22:23:41,294 [INFO] ✔ Completed in 0.0004391670227050781 seconds
2025-10-19 22:23:41,298 [INFO] General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-19 22:23:41,300 [INFO] ✔ Completed in 0.0005481243133544922 seconds
2025-10-19 22:23:41,309 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-19 22:23:41,311 [INFO] ✔ Completed in 0.0005319118499755859 seconds
2025-10-19 22:23:41,316 [INFO] General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-19 22:23:41,318 [INFO] ✔ Completed in 0.00043892860412597656 seconds
2025-10-19 22:23:41,320 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-19 22:23:41,321 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, y)
2025-10-19 22:23:41,323 [INFO]

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [10]:
m.gwp.emit.opt()

2025-10-19 22:23:41,444 [INFO] Mapping emit: (gwp, l, y, capacity, wf) → (gwp, l, y)
2025-10-19 22:23:41,450 [INFO] ✔ Completed in 0.000s
2025-10-19 22:23:41,453 [INFO] Mapping emit: (gwp, l, y, capacity, pv) → (gwp, l, y)
2025-10-19 22:23:41,457 [INFO] ✔ Completed in 0.000s
2025-10-19 22:23:41,460 [INFO] Mapping emit: (gwp, l, y, invcapacity, power.lii) → (gwp, l, y)
2025-10-19 22:23:41,465 [INFO] ✔ Completed in 0.000s
2025-10-19 22:23:41,474 [INFO] Mapping emit: (gwp, l, y, consume, cement) → (gwp, l, y)
2025-10-19 22:23:41,478 [INFO] ✔ Completed in 0.000s
2025-10-19 22:23:41,481 [INFO] Generating Program(example3).mps
2025-10-19 22:23:41,517 [INFO] Creating gurobi model for Program(example3)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file Program(example3).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE3): 95 rows, 87 columns, 217 nonzeros


2025-10-19 22:23:41,554 [INFO] Optimizing Program(example3) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 95 rows, 87 columns and 217 nonzeros
Model fingerprint: 0x39a95d48
Variable types: 84 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+06]
Presolve removed 83 rows and 76 columns
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 38 nonzeros
Variable types: 11 continuous, 0 integer (0 binary)

Root relaxation: objective 4.752148e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    475214.81481 475214.815  0.00%     -    0s

Explored 1 

2025-10-19 22:23:41,657 [INFO] Solution found. Use .sol() to display it
2025-10-19 22:23:41,663 [INFO] Creating Solution object, check.solution


In [11]:
len(m.variables)

87

In [12]:
m.show()

# Mathematical Program for Program(example3)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [13]:
m.sol(False)

# Solution for Program(example3)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>